<a href="https://colab.research.google.com/github/Kewton/kewton.blog.colab/blob/master/%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0%E3%82%B5%E3%83%B3%E3%83%97%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# スクレイピング基底クラスの定義

In [2]:
from bs4 import BeautifulSoup
import requests
import itertools
import time
from tqdm import notebook as tqdm

class BaseScraping:
  '''
  スクレイピングを実行する基底クラスです。
  _url_list に格納された URL を実行して html ドキュメントを取得し
  htmldoc2data を使用して必要なデータを取得します。

  Note:
    抽象メソッド speedbuffer と htmldoc2data を継承して使用してください。
  '''

  HEADERS = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
  }

  def __init__(self, _url_list:list):
    self._url_list = _url_list

  def run(self):
    results = {}
    for _url in tqdm.tqdm(self._url_list):
      print(_url)
      
      try:
        _data = self.htmldoc2data(self.get_html_doc(_url))
        results[_url] = _data
      except Exception as e:
        print("例外args:", e.args)
      
      self.speedbuffer()
    return results

  def speedbuffer(self):
    _sleeptime = 3
    print("wait " + str(_sleeptime) + "[sec]")
    time.sleep(_sleeptime)
    return

  def get_html_doc(self, _url):
    html_doc = requests.get(_url, headers=self.HEADERS, stream=True).content
    return html_doc

  def htmldoc2data(self, _html_doc):
    '''
    継承してください
    '''
    return _html_doc

# table 要素を DtaFrame に格納するクラスの定義

In [3]:
import re
from bs4 import BeautifulSoup
import pandas as pd


class GetTableDataFromWeb(BaseScraping):
  '''
  スクレイピングのサンプルクラスです。
  テーブルに格納されている値を取得します
  '''

  def __init__(self, _url_list:list, _css_selector):
    '''
    _css_selector：ターゲットとなるテーブル要素のcssセレクタを指定します
    '''
    super().__init__(_url_list)
    self.__css_selector= _css_selector

  def remove_blank_line_and_blank(self, _text):
    # 空白行削除
    text = _text.strip()
    text = re.sub(r'\n\n','\n',text)

    # 間の空白を削除
    text = re.sub(r'[ ]*','',text)
    
    return text

  def htmldoc2data(self, _html_doc):
    soup = BeautifulSoup(_html_doc, "html.parser")

    # テーブルを取得
    _table = soup.select_one(self.__css_selector)

    # theadタグを探す
    thead = _table.find('thead') 

    # tbodyタグを探す
    tbody = _table.find('tbody') 

    if thead is not None and tbody is not None:
      # パターン①：thead と tbody タグが存在する場合

      # ヘッダーを取得
      ths = thead.tr.find_all('th')

      # DataFrame のヘッダーを作成
      columns = []
      for th in ths:
        columns.append(th.text)
      df = pd.DataFrame(columns=columns)

      # body を取得
      trs = tbody.find_all('tr')
      for tr in trs:
        # 各レコードからテキストを取得し、空白行や余計な空白を削除してDataFrame に格納して返却
        tdlist = []
        for td in tr.find_all('td'):
          tdlist.append(self.remove_blank_line_and_blank(td.text))
        sr = pd.Series(tdlist, index=df.columns)
        df = df.append(sr, ignore_index=True)
      return df
    else:
      # パターン②：thead と tbody タグのいずれかが存在しない場合

      df = pd.DataFrame()
      trs = _table.find_all('tr')

      for tr in trs:
        # 各レコードからテキストを取得し、空白行や余計な空白を削除してDataFrame に格納して返却
        tdlist = []
        for td in tr.find_all('th'):
          tdlist.append(self.remove_blank_line_and_blank(td.text))

        for td in tr.find_all('td'):
          tdlist.append(self.remove_blank_line_and_blank(td.text))

        if len(tdlist) > 0:
          sr = pd.Series(tdlist)
          df = df.append(sr, ignore_index=True)
      return df
  

# サンプルクラスの実行

## フェブラリーステークス

In [16]:
# フェブラリーステークスの結果
_url_list =[
  "https://keiba.yahoo.co.jp/race/result/2205010811/",
  "https://keiba.yahoo.co.jp/race/result/2105010811/",
  "https://keiba.yahoo.co.jp/race/result/2005010811/"
]

getTableDataFromWeb = GetTableDataFromWeb(_url_list, "#raceScore")

results = getTableDataFromWeb.run()

print("============================")
print(_url_list[0])
results[_url_list[0]].head(5)

  0%|          | 0/3 [00:00<?, ?it/s]

https://keiba.yahoo.co.jp/race/result/2205010811/
wait 3[sec]
https://keiba.yahoo.co.jp/race/result/2105010811/
wait 3[sec]
https://keiba.yahoo.co.jp/race/result/2005010811/
wait 3[sec]
https://keiba.yahoo.co.jp/race/result/2205010811/


,着順,枠番,馬番,馬名性齢⁄馬体重⁄B,タイム(着差),通過順位上3Fタイム,騎手,人気(オッズ),調教師
0,1,3,6,カフェファラオ\n牡5/526(+6)/,1.33.8,04-0334.3,福永祐一\n57.0,2(5.1),堀宣行
1,2,8,15,テイエムサウスダン\n牡5/546(-8)/,1.34.221/2馬身,01-0135.0,岩田康誠\n57.0,5(8.9),飯田雄三
2,3,6,11,ソダシ\n牝4/476(+6)/,1.34.31/2馬身,02-0234.9,吉田隼人\n55.0,4(8.2),須貝尚介
3,4,7,13,ソリストサンダー\n牡7/486(0)/,1.34.3クビ,05-0634.6,戸崎圭太\n57.0,6(9),高柳大輔
4,5,4,7,タイムフライヤー\n牡7/482(+2)/,1.34.511/4馬身,10-1034.6,横山武史\n57.0,15(61.6),橋口慎介


In [18]:
# Jリーグ（J1, J2）の順位
_url_list =[
  "https://soccer.yahoo.co.jp/jleague/category/j1/standings",
  "https://soccer.yahoo.co.jp/jleague/category/j2/standings"
]

getTableDataFromWeb = GetTableDataFromWeb(_url_list, "#stand")

results = getTableDataFromWeb.run()

print("============================")
print(_url_list[0])
results[_url_list[0]].head(5)

  0%|          | 0/2 [00:00<?, ?it/s]

https://soccer.yahoo.co.jp/jleague/category/j1/standings
wait 3[sec]
https://soccer.yahoo.co.jp/jleague/category/j2/standings
wait 3[sec]
https://soccer.yahoo.co.jp/jleague/category/j1/standings


,0,1,2,3,4,5,6,7,8,9
0,順位,チーム名,勝点,試合数,勝数,引分数,敗数,得点,失点,得失点差
1,1,横浜F・マリノス,10,5,3,1,1,11,7,4
2,2,川崎フロンターレ,10,5,3,1,1,9,7,2
3,3,柏レイソル,6,3,2,0,1,5,2,3
4,4,鹿島アントラーズ,6,3,2,0,1,4,3,1


In [19]:
# 海外サッカーの順位
_url_list =[
  "https://soccer.yahoo.co.jp/ws/standings/52",
  "https://soccer.yahoo.co.jp/ws/standings/53",
  "https://soccer.yahoo.co.jp/ws/standings/67"
]

getTableDataFromWeb = GetTableDataFromWeb(_url_list, "table")

results = getTableDataFromWeb.run()

print("============================")
print(_url_list[0])
results[_url_list[0]].head(5)

  0%|          | 0/3 [00:00<?, ?it/s]

https://soccer.yahoo.co.jp/ws/standings/52
wait 3[sec]
https://soccer.yahoo.co.jp/ws/standings/53
wait 3[sec]
https://soccer.yahoo.co.jp/ws/standings/67
wait 3[sec]
https://soccer.yahoo.co.jp/ws/standings/52


,順位,チーム名,勝点,試合数,勝数,引分数,負数,得点,失点,得失点差
0,1,マンチェスター・シティ,69,28,22,3,3,68,18,50
1,2,リヴァプール,63,27,19,6,2,71,20,51
2,3,チェルシー,53,26,15,8,3,53,18,35
3,4,アーセナル,48,25,15,3,7,41,29,12
4,5,マンチェスター・ユナイテッド,47,28,13,8,7,45,38,7
